In [6]:
from Bio import SeqIO
from Bio.Seq import MutableSeq
from Bio.Alphabet import generic_alphabet
import copy
import pandas as pd


# filename input. paste your filename with extension in here. extension to make things explicitly
filename='test_align_simple.fasta'

# parsing fasta file and creating list out of it. 
# TOTRY: is it possible to make iterator, not list?
records = list(SeqIO.parse(filename, "fasta", generic_alphabet))


# MutableSeq is an editable sequence object (with an alphabet).
# Unlike normal python strings and our basic sequence object (the Seq class) 
# which are immutable, the MutableSeq lets you edit the sequence in place. 
# However, this means you cannot use a MutableSeq object as a dictionary key.
rec3 = [MutableSeq(str(q.seq)) for q in records]

# empty list by length of the sequence
# each nested list is the nucleotide position
# it will contain the integers which are the lengths of the deletions in that position found in reads
# the number of integers is the quantity of deletions which start at this position
rec3ins = [[] for i in range(len(rec3[0]))]

# empty list by length of the sequence
# each nested list is the nucleotide position
# it will contain the integers which are the lengths of the insertions in that position found in reads
# the number of integers is the quantity of insertions which start at this position
rec3dels = [[] for i in range(len(rec3[0]))]

# loop through the list of reads. i.e. total number of reads

for j in range(len(rec3)):   # 'rec3' is an array of sequences
 
    if rec3[j] != rec3[0]: # if rec[0] is rec[0], loop again. As rec[0] is the reference ('denoted as : entered manually')
#       print(j,len(rec3))
        cnt2 = 0
    
   
        for i in range(len(rec3[j])):  # take sequence length
            if i < cnt2:                 # it's here to iterate without operation till the end of deletion or insertion
                # rec3[j] is sequence
                continue               # i is a number of nucleotide
            
            if rec3[j][i] != rec3[0][i]:  # if nucleotide in sequence under study is not equal to nucl in reference
                                          # which is sequence under [0] index
                                          # check up whether it is a gap (as it's may be just another nucl)
                
                ###################
                # this block counts deletions:
                ####################
                if rec3[j][i] == '-':     # if nucl in seq  under study is gap 
                    cnt = 0
                    cnt2 = 0
                    
                    # we count the length of deletion:
                    for k in range(i + 1, len(rec3[0])):   # here's index shift to go further, next after gap
                                                           # going throught the deletion length, trying to find its end
                        if rec3[j][k] == '-' and rec3[j][k] != rec3[0][k]:
                            cnt += 1  
                        
                        if rec3[j][k] != '-':    # if nucl is not gap, break loop, as the deletion is over
                            cnt2 = k
                            break
                    
                    # append the resulting number (length of the deletion) at the corresponding index position in the container
                    rec3dels[i].append(cnt + 1) # add one more, because w  
                    
                
                
                ###################
                # this block counts insertions:
                ####################
                if rec3[0][i] == '-' and rec3[j][i] != 'N': # if nuc in ref is gap (cons?) and nuc in seq isn't 'N'
                    cnt = 0
                    cnt2 = 0
                    
                    for k in range(i + 1, len(rec3[0])):
                        if rec3[0][k] == '-' and rec3[j][k] != rec3[0][k]:
                            cnt += 1
                        if rec3[0][k] != '-':
                            cnt2 = k
                            break
                    rec3ins[i].append(cnt+1)
#                   i=cnt2


        
print("rec3dels: ", rec3dels, end="\n")      
print("rec3dels: ", rec3ins, end="\n")


rec3ins2 = copy.deepcopy(rec3ins)
rec3dels2 = copy.deepcopy(rec3dels)
cons = copy.deepcopy(rec3[0])

i = 1

while i < len(cons):
    if cons[i] == '-':
        if rec3ins2[i] != []:  # if nested list under i index is empty
            rec3ins2[i - 1].extend(copy.deepcopy(rec3ins2[i]))  # take previous nested list
        del rec3ins2[i]
        
        if rec3dels2[i] != []:
            rec3dels2[i - 1].extend(copy.deepcopy(rec3dels2[i]))
            
        del rec3dels2[i]
        del cons[i]
        
    else:
        i += 1


        
df=pd.DataFrame(rec3dels2).T
#with pd.option_context('display.max_rows', None, 'display.max_columns', df.shape[1]):
#    display(df)

df2=pd.DataFrame(rec3ins2).T
#with pd.option_context('display.max_rows', None, 'display.max_columns', df2.shape[1]):
#    display(df2)

writer = pd.ExcelWriter(filename+'.xlsx')
df.to_excel(writer,'dels')
df2.to_excel(writer,'ins')

writer.save()


                    
                    








rec3dels:  [[], [], [], [], [], [], [], [], [], [], [], [], [6, 6], [], [], [], [], [], [], [], [], []]
rec3dels:  [[], [], [], [], [4, 4], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [2]:
from Bio import SeqIO
data = SeqIO.parse("DK-M12_S125_R1_001 assembled to St4 - St4.fasta", "fasta")

In [10]:
data

<generator object parse at 0x000001B79C92CFC0>

In [ ]:
# I believe, this is the way to rewrite the script
# just use the AlignIO array structure to deal with the alignment

In [8]:
from Bio import AlignIO

sam_as_align = AlignIO.read("DK-M12_S125_R1_001 assembled to St4 - St4.fasta", "fasta")

In [9]:
sam_as_align

<<class 'Bio.Align.MultipleSeqAlignment'> instance (237749 records of length 125, SingleLetterAlphabet()) at 197c381bbe0>

In [10]:
print(sam_as_align)

SingleLetterAlphabet() alignment with 237749 rows and 125 columns
GC-C-TCAG-CCCG-TTT-CT-CC-T--G-G--CT-C--A-GTT...GTG St4
ANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:1:1208:14854:89781_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:2:1208:13878:70129_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:2:2208:21158:100448_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:1:2106:6722:41193_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:2:1105:7409:37190_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:2:1108:1780:68192_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN C00130:45:HWTNLBCXY:2:2109:12948:76750_1:N:0:GCGTAGTA+TAGATCGC_(reversed)
GANNNN

In [11]:
nucls = ["A", "T", "G", "C"]

In [16]:
sam_as_align[0]

SeqRecord(seq=Seq('GC-C-TCAG-CCCG-TTT-CT-CC-T--G-G--CT-C--A-GTT----------...GTG', SingleLetterAlphabet()), id='St4', name='St4', description='St4 A new nucleotide sequence entered manually', dbxrefs=[])

In [17]:
sam_as_align[1]

SeqRecord(seq=Seq('ANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet()), id='C00130:45:HWTNLBCXY:1:1208:14854:89781_1:N:0:GCGTAGTA+TAGATCGC_(reversed)', name='C00130:45:HWTNLBCXY:1:1208:14854:89781_1:N:0:GCGTAGTA+TAGATCGC_(reversed)', description='C00130:45:HWTNLBCXY:1:1208:14854:89781_1:N:0:GCGTAGTA+TAGATCGC_(reversed)', dbxrefs=[])

In [19]:
sam_as_align[:, 0]

...


In [22]:
len(sam_as_align)  # alignment length is its number of rows (here - number of reads)

237749

In [23]:
sam_as_align.get_alignment_length()

125

In [45]:
ref_seq

SeqRecord(seq=Seq('AAAAGGGGGGAAAA', SingleLetterAlphabet()), id='ref', name='ref', description='ref', dbxrefs=[])

In [183]:
from Bio.Seq import Seq

sam_as_align = AlignIO.read("test_align_simple.fasta", "fasta")
ref_seq = sam_as_align[0]



dels_container = [[] for k in range(0, len(ref_seq))]
ins_container = [[] for x in range(0, len(ref_seq))]


#dels_container = {k:() for (k) in  range(0, len(ref_seq))}
#ins_container = {k:() for (k) in  range(0, len(ref_seq))}


nucs = ["A", "T", "G", "C"]
print("i'm here")


for seq_ind in range(1, len(sam_as_align)): # create loop through alignment rows (index)
    start_nuc = 0
    
    
    for nuc in range(sam_as_align.get_alignment_length()):  # create loop through nucleotides: alignment columns
        if nuc < start_nuc:
            continue
        
        if sam_as_align[seq_ind, nuc] == ref_seq[nuc] or sam_as_align[seq_ind, nuc] == "N":

            continue
          
        elif ref_seq[nuc] != sam_as_align[seq_ind, nuc]:

            
            if ref_seq[nuc] in nucs and sam_as_align[seq_ind, nuc] == "-":
                for nuc_del in range(nuc + 1, len(ref_seq)):
                    if ref_seq[nuc_del] in nucs and sam_as_align[seq_ind, nuc_del] == "-":
                        dels += 1
                    else:
                        dels_container[nuc].append(dels + 1)
                        start_nuc = nuc_del 
                        dels = 0
                        print("i count dels")
                        break
                        
            elif ref_seq[nuc] == "-" and sam_as_align[seq_ind, nuc] in nucs:
                
                for nuc_del in range(nuc + 1, len(ref_seq)):
                    if ref_seq[nuc_del] == "-" and sam_as_align[seq_ind, nuc_del] in nucs:
                        ins += 1
                    else:
                        ins_container[nuc].append(ins + 1)

                        start_nuc = nuc_del 
                        ins = 0
                        print("i count ins")
                        break


i'm here
i count ins
i count dels


In [184]:
print(dels_container)
print(ins_container)

[[], [], [], [], [], [], [], [], [], [], [], [], [6], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [7], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [132]:
table = pd.DataFrame(dels_container).T

In [133]:
table

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124


In [129]:
sam_as_align = list(SeqIO.parse("DK-M12_S125_R1_001 assembled to St4 - St4.fasta", "fasta"))


ref_seq = sam_as_align[0]

dels_container = [[] for x in range(0, len(ref_seq))]
ins_container = [[] for x in range(0, len(ref_seq))]
container = []


nucs = ["A", "T", "G", "C"]
print("i'm here")


for seq_ind in range(1, len(sam_as_align)): # create loop through alignment rows (index)
    start_nuc = 0
    
    
    for nuc in range(sam_as_align.get_alignment_length()):  # create loop through nucleotides: alignment columns
        if nuc < start_nuc:
            continue
        
        if sam_as_align[seq_ind, nuc] == ref_seq[nuc] or sam_as_align[seq_ind, nuc] == "N":
            continue
          
        elif ref_seq[nuc] != sam_as_align[seq_ind, nuc]:

            
            if ref_seq[nuc] in nucs and sam_as_align[seq_ind, nuc] == "-":
                
                for nuc_del in range(nuc + 1, len(ref_seq)):
                    if ref_seq[nuc_del] in nucs and sam_as_align[seq_ind, nuc_del] == "-":
                        dels += 1
                    else:
                        dels_container[nuc].append(dels + 1)
                        
                        start_nuc = nuc_del 
                        dels = 0
                        break
            
                        
            elif ref_seq[nuc] == "-" and sam_as_align[seq_ind, nuc] in nucs:
                
                for nuc_del in range(nuc + 1, len(ref_seq)):
                    if ref_seq[nuc_del] == "-" and sam_as_align[seq_ind, nuc_del] in nucs:
                        ins += 1
                    else:
                        ins_container[nuc].append(ins + 1)
                        
                        start_nuc = nuc_del 
                        ins = 0
                        break
                           

i'm here


AttributeError: 'list' object has no attribute 'get_alignment_length'

In [ ]:

print(dels_container)
print(ins_container)


def find_indels()
    sam_as_align = AlignIO.read("test_align.fasta", "fasta")


    ref_seq = sam_as_align[0]

    dels_container = [[] for x in range(0, len(ref_seq))]
    ins_container = [[] for x in range(0, len(ref_seq))]
    container = []

    nucs = ["A", "T", "G", "C"]
    for seq_ind in range(1, len(sam_as_align)): # create loop through alignment rows (index)
        start_nuc = 0
    

        for nuc in range(sam_as_align.get_alignment_length()):  # create loop through nucleotides: alignment columns


            if sam_as_align[seq_ind, nuc] == ref_seq[nuc] or sam_as_align[seq_ind, nuc] == "N":
                continue

            elif ref_seq[nuc] != sam_as_align[seq_ind, nuc]:
                
                

            
            
            
            
            
            
if ref_seq[nuc] in nucs and sam_as_align[seq_ind, nuc] == "-":
                
    for nuc_del in range(nuc + 1, len(ref_seq)):
        print(nuc_del)
        if ref_seq[nuc_del] in nucs and sam_as_align[seq_ind, nuc_del] == "-":
            dels += 1
        else:
            dels_container[nuc].append(dels + 1)
                        
            start_nuc = nuc_del 
            dels = 0break
            
            
            elif ref_seq[nuc] == "-" and sam_as_align[seq_ind, nuc] in nucs:
                
                for nuc_del in range(nuc + 1, len(ref_seq)):
                    print(nuc_del)
                    if ref_seq[nuc_del] == "-" and sam_as_align[seq_ind, nuc_del] in nucs:
                        ins += 1
                    else:
                        ins_container[nuc].append(ins + 1)
                        
                        start_nuc = nuc_del 
                        ins = 0
                        break
                           

In [115]:
print(dels_container)
print(ins_container)

[[], [], [], [], [6], [5], [4], [3], [2], [1], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [4], [3], [2], [1], [], [], [], [], [], [], [], []]


In [99]:
len(dels_container)
                    

26

In [71]:
len("AAAAGGGGGGAAAA----TTTTAAAA")

26

In [4]:
import pandas as pd


In [6]:
counter = 0
df = pd.DataFrame()

for record in data:
    sequence = list(record.seq)
    seq_name = record.id
    ser = pd.Series(name=seq_name, data=sequence)
    df = df.append(ser, ignore_index=True)
    

KeyboardInterrupt: 

In [23]:
df

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,G,C,-,C,-,T,C,A,G,-,...,T,-,-,T,-,C,A,G,T,G


In [28]:
len(df.loc[0, :])

125

In [29]:
s = "ANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN"

In [34]:
ref

'GC-C-TCAG-CCCG-TTT-CT-CC-T--G-G--CT-C--A-GTT------------------------------------------------T-A-CTAG--T-GCCA-TTT--GT--T-CAGTG'

In [33]:
ref = "GC-C-TCAG-CCCG-TTT-CT-CC-T--G-G--CT-C--A-GTT------------------------------------------------T-A-CTAG--T-GCCA-TTT--GT--T-CAGTG"

In [10]:
len(rec3)

237749

In [21]:
type(cnt2)

int

In [29]:
b = 0

if b == 0:
    print("hey")
elif b != 1:
    print("hey hey")

hey
